<a href="https://colab.research.google.com/github/cristhiansito123/MLDS6project/blob/entrega3/scripts/training/mainModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


import neccesary packages

In [2]:
import json
import requests
import os
import shutil
import time
import sys

import numpy as np
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline

#Carga de datos

In [4]:

metadata_path = '/content/drive/MyDrive/MLDS6project/data.csv'

metadata = pd.read_csv(metadata_path, sep=',')


In [5]:
metadata

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


# Modelo

In [6]:
mapeo = {'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5}

# Aplicar el mapeo a la columna del DataFrame
metadata['type_numeric'] = metadata['type'].map(mapeo)

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

In [52]:
feature_names = ['step', 'type_numeric', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
X = metadata[feature_names].values
y = metadata['isFraud'].values

Se reduce el número de datos a 1000000.

In [53]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
X_resampled_downsized, y_resampled_downsized = resample(X, y, n_samples=1000000)

In [54]:
# Escala las características
scaler = StandardScaler()
X = scaler.fit_transform(X_resampled_downsized)
y = y_resampled_downsized

In [55]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [56]:
# Configura los parámetros del modelo
input_shape = (X_train.shape[1], 1)
filters = 32
kernel_size = 3
pool_size = 2
lstm_units = 64
dropout_rate = 0.25
dense_units = 1

# Crea el modelo
model = Sequential()
model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(units=lstm_units))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=dense_units, activation='sigmoid'))

## Compilación del modelo

In [57]:
# Compila el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrena el modelo
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
25000/25000 [==============================] - 129s 5ms/step - loss: 0.0068 - accuracy: 0.9988 - val_loss: 0.0046 - val_accuracy: 0.9992
Epoch 2/10
25000/25000 [==============================] - 131s 5ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0036 - val_accuracy: 0.9993
Epoch 3/10
25000/25000 [==============================] - 125s 5ms/step - loss: 0.0038 - accuracy: 0.9991 - val_loss: 0.0033 - val_accuracy: 0.9993
Epoch 4/10
25000/25000 [==============================] - 132s 5ms/step - loss: 0.0035 - accuracy: 0.9992 - val_loss: 0.0032 - val_accuracy: 0.9992
Epoch 5/10
25000/25000 [==============================] - 124s 5ms/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 0.0028 - val_accuracy: 0.9993
Epoch 6/10
25000/25000 [==============================] - 132s 5ms/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 0.0027 - val_accuracy: 0.9993
Epoch 7/10
25000/25000 [==============================] - 124s 5ms/step - loss: 0.0031 - accuracy: 0.9992 - val_

Evaluación del modelo

In [58]:
# Evalúa el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

6250/6250 [==============================] - 17s 3ms/step - loss: 0.0027 - accuracy: 0.9993
Loss: 0.0026551962364465
Accuracy: 0.9993000030517578


## Métricas de desempeño

Usamos un 10% aleatorio de nuestro dataframe original para medir la calidad de nuestro modelo:

In [59]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

percent = 0.1  # Proporción del 10%
datos_prueba = metadata.sample(frac=percent, random_state=42)
X_prueba = datos_prueba[feature_names].values
y_prueba = datos_prueba['isFraud'].values

# Realiza las predicciones en el conjunto de prueba
y_pred = model.predict(X_prueba)



19884/19884 [==============================] - 45s 2ms/step


In [60]:
y_pred = np.where(y_pred > 0.5, 1, 0) ##Convertir de float a int

In [61]:
y_pred = y_pred.ravel()

In [62]:
# Calcula las métricas de desempeño
accuracy = accuracy_score(y_prueba, y_pred)
precision = precision_score(y_prueba, y_pred)
recall = recall_score(y_prueba, y_pred)
f1 = f1_score(y_prueba, y_pred)

print("Exactitud:", accuracy)
print("Precisión:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Exactitud: 0.9074956543059306
Precisión: 0.011924383598506508
Recall: 0.8678090575275398
F1-score: 0.023525508087930316
